# **Without speech**

In [2]:
!pip install -q sacrebleu unbabel-comet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 38.7 MB/s eta 0:00:00


# Phi

In [5]:
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu
from sacrebleu.metrics import BLEU
from comet import download_model, load_from_checkpoint

# Load CSVs
french_df = pd.read_csv('french.csv')
translations_df = pd.read_csv('phi_translations.csv')

# Ensure alignment of rows
references = french_df['french_text'].tolist()
translations = translations_df['translated_french'].tolist()

# BLEU Score Calculation
def calculate_bleu(references, translations):
    references = [[ref.split()] for ref in references]  # Tokenize references
    translations = [trans.split() for trans in translations]  # Tokenize translations
    bleu_score = corpus_bleu(references, translations)  # nltk BLEU
    return bleu_score



# # Calculate BLEU
bleu_score = calculate_bleu(references, translations)


print(f"BLEU Score: {bleu_score}")



BLEU Score: 0.024297354013372036


In [6]:
# def calculate_comet(references, translations):
model_path = download_model("Unbabel/wmt20-comet-da")
model = load_from_checkpoint(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/437 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.54k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [ ]:
data = [
    {"src": "", "mt": trans, "ref": ref}  # COMET requires source, mt, ref
    for trans, ref in zip(translations, references)
]
comet_scores = model.predict(data, batch_size=8, gpus=1)

print(comet_scores)

# Extract the individual scores from the Prediction object
individual_scores = comet_scores[0]  # Get the scores (first element of the tuple)
print(individual_scores)

comet_score = sum(individual_scores) / len(individual_scores) if individual_scores else 0

# Compute the mean COMET score
print(f"COMET Score: {comet_score}")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 32/32 [00:12<00:00,  2.56it/s]


Prediction([('scores', [0.16255095601081848, -1.4700634479522705, -1.582566499710083, -1.582566499710083, -0.008525950834155083, -1.3845375776290894, -0.6815034747123718, -0.6815034747123718, -1.340512990951538, -1.4097816944122314, -1.489426851272583, -1.489426851272583, -0.4724949598312378, -1.7516701221466064, -1.5970404148101807, 1.141540288925171, -1.2602441310882568, -1.634439468383789, -1.634439468383789, -0.6797480583190918, -1.6285781860351562, -1.0473918914794922, -1.5406672954559326, 1.1006505489349365, 0.6505517959594727, -0.32352274656295776, 1.0192394256591797, -1.0831576585769653, -1.1391608715057373, -1.3852328062057495, -1.3852328062057495, -1.3852328062057495, -1.38523268699646, -1.5164433717727661, -1.5164433717727661, -1.9111673831939697, -1.2254728078842163, -1.2254728078842163, -1.2254728078842163, -1.2254728078842163, -1.6971547603607178, 0.5547471046447754, -1.466651201248169, -1.4984604120254517, -1.6243873834609985, -1.6263302564620972, -1.6263302564620972, -1

**Phi** \

Before Speech \
Bleu: 0.024297354013372036 \
Comet: -1.0173832897171378\

# Qwen

In [ ]:
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu
from sacrebleu.metrics import BLEU
from comet import download_model, load_from_checkpoint

# Load CSVs
french_df = pd.read_csv('french.csv')
translations_df = pd.read_csv('qwen_translations.csv')

# Ensure alignment of rows
qwen_references = french_df['french_text'].tolist()
qwen_translations = translations_df['translated_french'].tolist()




# # Calculate BLEU
bleu_score = calculate_bleu(qwen_references, qwen_translations)


print(f"BLEU Score: {bleu_score}")



BLEU Score: 0.03915828215265459


In [ ]:
data = [
    {"src": "", "mt": trans, "ref": ref}  # COMET requires source, mt, ref
    for trans, ref in zip(qwen_translations, qwen_references)
]
comet_scores = model.predict(data, batch_size=8, gpus=1)

print(comet_scores)

# Extract the individual scores from the Prediction object
individual_scores = comet_scores[0]  # Get the scores (first element of the tuple)
print(individual_scores)

comet_score = sum(individual_scores) / len(individual_scores) if individual_scores else 0

# Compute the mean COMET score
print(f"COMET Score: {comet_score}")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 32/32 [00:07<00:00,  4.04it/s]


Prediction([('scores', [0.6045781373977661, -1.5687804222106934, -1.5265557765960693, -1.5265557765960693, -1.2964216470718384, 0.04065784811973572, -0.3734416961669922, -0.3734416961669922, -1.3610221147537231, -1.470866322517395, -1.0671420097351074, -1.0671420097351074, -0.7569865584373474, 0.18476752936840057, -1.1620116233825684, 1.177179217338562, -1.8970478773117065, -1.1102159023284912, -1.1102159023284912, -1.853381633758545, -1.3353219032287598, -1.8657002449035645, -1.4462575912475586, -1.9782435894012451, -0.9105145335197449, -1.1327495574951172, -1.3253583908081055, -1.1679000854492188, -1.468625545501709, -1.516753911972046, -1.516753911972046, -1.516753911972046, -1.5167537927627563, -1.5691648721694946, -1.5691648721694946, -1.4785678386688232, -1.7677245140075684, -1.7677245140075684, -1.7677245140075684, -1.7677245140075684, -1.2137329578399658, 0.052629388868808746, -0.5557111501693726, -1.1428205966949463, -0.91587233543396, -1.8044633865356445, -1.8044633865356445,

**Qwen** \

Before Speech \
Bleu: 0.03915828215265459 \
Comet: -0.8780392711609601 \

# Mistral

In [ ]:
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu
from sacrebleu.metrics import BLEU
from comet import download_model, load_from_checkpoint

# Load CSVs
french_df = pd.read_csv('french.csv')
translations_df = pd.read_csv('mistral_translations.csv')

In [ ]:
translations_df

,translated_french
0,NaN
1,NaN
2,DIT FRANZ (literally: Franz said) in French Fr...
3,DIT FRANZ (literally: Franz said) in French Fr...
4,NaN
...,...
245,NaN
246,NaN
247,Vrai a dit Andrea (formal: Andrea a dit vrai)
248,EN FRANÇAIS : SOINGT FRANZ DOIT attendre


In [ ]:
import pandas as pd

# Replace NaN values with whitespace in the entire DataFrame
translations_df = translations_df.fillna(' ')


In [ ]:
# Ensure alignment of rows
mistral_references = french_df['french_text'].tolist()
mistral_translations = translations_df['translated_french'].tolist()

In [ ]:
len(mistral_translations)

250

In [ ]:
# # Calculate BLEU
bleu_score = calculate_bleu(mistral_references, mistral_translations)


print(f"BLEU Score: {bleu_score}")

BLEU Score: 0.04061247995012028


In [ ]:
data = [
    {"src": "", "mt": trans, "ref": ref}  # COMET requires source, mt, ref
    for trans, ref in zip(mistral_translations, mistral_references)
]
comet_scores = model.predict(data, batch_size=8, gpus=1)

print(comet_scores)

# Extract the individual scores from the Prediction object
individual_scores = comet_scores[0]  # Get the scores (first element of the tuple)
print(individual_scores)

comet_score = sum(individual_scores) / len(individual_scores) if individual_scores else 0

# Compute the mean COMET score
print(f"COMET Score: {comet_score}")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 32/32 [00:08<00:00,  3.60it/s]


Prediction([('scores', [-1.7180739641189575, -1.450758457183838, -1.576219081878662, -1.576219081878662, -1.3643895387649536, -1.087174415588379, -1.5205183029174805, -1.5205183029174805, -1.6446537971496582, -0.1913941353559494, -1.336023211479187, -1.336023211479187, -1.3591140508651733, -1.2870593070983887, -1.5742824077606201, -1.4185295104980469, -1.1790461540222168, -0.9076386094093323, -0.9076386094093323, -1.4931659698486328, -1.3979816436767578, -1.313454508781433, -1.4824295043945312, 0.06018570810556412, -0.6985795497894287, -0.15442268550395966, -1.419508457183838, -2.126554489135742, -1.4704022407531738, -1.6404823064804077, -1.6404823064804077, -1.6404823064804077, -1.6404824256896973, -1.489853024482727, -1.489853024482727, -1.2521240711212158, -0.9713106751441956, -0.9713106751441956, -0.9713106751441956, -0.9713106751441956, -1.1935685873031616, 0.8949620127677917, -1.6471633911132812, -0.9317358136177063, -0.9853330254554749, -1.5095324516296387, -1.5095324516296387, 

**Mistral** \

Before Speech \
Bleu: 0.04061247995012028 \
Comet: -1.2252497419714927 \

# **With speech**

# phi

In [33]:
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu
from sacrebleu.metrics import BLEU
from comet import download_model, load_from_checkpoint

# Load CSVs
french_df = pd.read_csv('french.csv')
translations_df = pd.read_csv('phi_ser_translations.csv')
# translations_df

# Ensure alignment of rows
phi_references = french_df['french_text'].tolist()
phi_translations = translations_df['translated_french'].tolist()

# # Calculate BLEU
bleu_score = calculate_bleu(phi_references, phi_translations)


print(f"BLEU Score: {bleu_score}")

BLEU Score: 0.040262620394878615


In [34]:
data = [
    {"src": "", "mt": trans, "ref": ref}  # COMET requires source, mt, ref
    for trans, ref in zip(phi_translations, phi_references)
]
comet_scores = model.predict(data, batch_size=8, gpus=1)

print(comet_scores)

# Extract the individual scores from the Prediction object
individual_scores = comet_scores[0]  # Get the scores (first element of the tuple)
print(individual_scores)

comet_score = sum(individual_scores) / len(individual_scores) if individual_scores else 0

# Compute the mean COMET score
print(f"COMET Score: {comet_score}")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 32/32 [00:06<00:00,  4.65it/s]


Prediction([('scores', [-0.14843879640102386, -1.184540033340454, -1.4069223403930664, -1.4069223403930664, 0.6712198853492737, -1.1068530082702637, 0.18771111965179443, 0.18771111965179443, -1.186919927597046, -1.3111474514007568, -1.4049973487854004, -1.4049973487854004, 0.5915156006813049, -1.4742637872695923, -0.4972143769264221, -1.9440913200378418, -1.4792143106460571, -0.5009995102882385, -0.5009995102882385, -1.474631428718567, -1.3310436010360718, -1.1258963346481323, 0.6506603360176086, -1.506561040878296, -1.7128589153289795, -1.6682006120681763, -0.8669578433036804, -1.9282705783843994, -1.6612939834594727, -1.5946528911590576, -1.5946528911590576, -1.5946528911590576, -1.5946530103683472, -1.2787995338439941, -1.2787995338439941, 0.9949560761451721, -0.8394457101821899, -0.8394457101821899, -0.8394457101821899, -0.8394457101821899, -0.6580488681793213, -1.736046314239502, 0.8550844788551331, -0.9451209902763367, -1.4511715173721313, -1.4549356698989868, -1.4549356698989868

# Mistral

In [25]:
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu
from sacrebleu.metrics import BLEU
from comet import download_model, load_from_checkpoint

# Load CSVs
french_df = pd.read_csv('french.csv')
translations_df = pd.read_csv('mistral_ser_translations.csv')

In [27]:
# Ensure alignment of rows
mistral_references = french_df['french_text'].tolist()
mistral_translations = translations_df['translated_french'].tolist()

In [28]:
# # Calculate BLEU
bleu_score = calculate_bleu(mistral_references, mistral_translations)


print(f"BLEU Score: {bleu_score}")

BLEU Score: 0.057708685465438815


In [29]:
data = [
    {"src": "", "mt": trans, "ref": ref}  # COMET requires source, mt, ref
    for trans, ref in zip(mistral_translations, mistral_references)
]
comet_scores = model.predict(data, batch_size=8, gpus=1)

print(comet_scores)

# Extract the individual scores from the Prediction object
individual_scores = comet_scores[0]  # Get the scores (first element of the tuple)
print(individual_scores)

comet_score = sum(individual_scores) / len(individual_scores) if individual_scores else 0

# Compute the mean COMET score
print(f"COMET Score: {comet_score}")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 32/32 [00:07<00:00,  4.35it/s]


Prediction([('scores', [-1.4374284744262695, -1.4110090732574463, -1.472968578338623, -1.472968578338623, 0.38645169138908386, -1.4172450304031372, -1.3235396146774292, -1.3235396146774292, 1.1898751258850098, -1.3014510869979858, -1.470400094985962, -1.470400094985962, -1.4536799192428589, -1.4214210510253906, -0.7910163402557373, 1.1415404081344604, -1.1376768350601196, -1.6480027437210083, -1.6480027437210083, -1.411569356918335, -1.469534158706665, -1.5061534643173218, -1.3870104551315308, -1.5137253999710083, 0.7691469788551331, -0.18809708952903748, -1.4903883934020996, -1.0875771045684814, -1.5024915933609009, -1.2769075632095337, -1.2769075632095337, -1.2769075632095337, -1.2769074440002441, -1.2769074440002441, -1.2769074440002441, -1.2423686981201172, -1.2990285158157349, -1.2990285158157349, -1.2990285158157349, -1.2990285158157349, -1.6302509307861328, 0.812028169631958, -1.715638518333435, -1.5138421058654785, -1.4457499980926514, -0.39959925413131714, -0.39959925413131714

# qwen

In [35]:
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu
from sacrebleu.metrics import BLEU
from comet import download_model, load_from_checkpoint

# Load CSVs
french_df = pd.read_csv('french.csv')
translations_df = pd.read_csv('qwen_ser_translations.csv')
# translations_df

# Ensure alignment of rows
qwen_references = french_df['french_text'].tolist()
qwen_translations = translations_df['translated_french'].tolist()

# # Calculate BLEU
bleu_score = calculate_bleu(qwen_references, qwen_translations)


print(f"BLEU Score: {bleu_score}")

BLEU Score: 0.03454403240331334


In [36]:
data = [
    {"src": "", "mt": trans, "ref": ref}  # COMET requires source, mt, ref
    for trans, ref in zip(qwen_translations, qwen_references)
]
comet_scores = model.predict(data, batch_size=8, gpus=1)

print(comet_scores)

# Extract the individual scores from the Prediction object
individual_scores = comet_scores[0]  # Get the scores (first element of the tuple)
print(individual_scores)

comet_score = sum(individual_scores) / len(individual_scores) if individual_scores else 0

# Compute the mean COMET score
print(f"COMET Score: {comet_score}")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 32/32 [00:06<00:00,  4.96it/s]


Prediction([('scores', [-1.4374284744262695, -0.6212479472160339, -1.5498894453048706, -1.5498894453048706, -0.20115165412425995, -1.4885876178741455, -1.0148061513900757, -1.0148061513900757, -1.1869194507598877, -1.369230031967163, -1.4183794260025024, -1.4183794260025024, -1.4840898513793945, -1.460631012916565, -0.8935206532478333, -1.3616931438446045, -1.5738340616226196, -1.535886526107788, -1.535886526107788, -1.4048539400100708, -1.3543927669525146, -1.608460783958435, -1.2795860767364502, 0.9862115979194641, -1.4690485000610352, -1.443068027496338, 0.8741065859794617, -1.645490050315857, -1.4559638500213623, -1.5215373039245605, -1.5215373039245605, -1.5215373039245605, -1.5215370655059814, -1.4858057498931885, -1.4858057498931885, -1.1449888944625854, -1.5298181772232056, -1.5298181772232056, -1.5298181772232056, -1.5298181772232056, -1.5183041095733643, -1.6489372253417969, -1.65189790725708, 0.7692955136299133, -1.3342562913894653, -1.7688195705413818, -1.7688195705413818, 